In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [3]:
import cv2, pickle
import numpy as np
import tensorflow as tf
from sklearn.metrics.pairwise import cosine_similarity
from datetime import datetime
import pandas as pd

model = tf.keras.models.load_model("resnet_embedding_model.keras", compile=False)

with open("face_embeddings.pkl", "rb") as f:
    known_embeddings, known_labels = pickle.load(f)

known_embeddings = np.array(known_embeddings)

e:\miniconda3\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/attr_value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
e:\miniconda3\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/tensor.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
e:\miniconda3\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/resource_handle.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnin

In [4]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (128)                  │       262,272 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,849,984 (90.98 MB)

 Trainable params: 262,272 (1.00 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [5]:
face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)

In [6]:
def preprocess(face):
    face = cv2.resize(face, (224,224))
    face = face / 255.0
    return np.expand_dims(face, axis=0)

In [10]:
def recognize(face, threshold=0.7):
    
    emb = model.predict(preprocess(face), verbose=0)

    emb = emb.reshape(1, -1)

    known_embs = known_embeddings.reshape(known_embeddings.shape[0], -1)

    sims = cosine_similarity(emb, known_embs)
    idx = np.argmax(sims)

    if sims[0][idx] > threshold:
        return known_labels[idx], sims[0][idx]
    return "Unknown", sims[0][idx]

In [8]:
def mark_attendance(name):
    date = datetime.now().strftime("%Y-%m-%d")
    time = datetime.now().strftime("%H:%M:%S")

    try:
        df = pd.read_csv("attendance.csv")
        if ((df["Name"] == name) & (df["Date"] == date)).any():
            return
    except:
        df = pd.DataFrame(columns=["Name","Date","Time"])

    df.loc[len(df)] = [name, date, time]
    df.to_csv("attendance.csv", index=False)

In [11]:
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x,y,w,h) in faces:
        face = frame[y:y+h, x:x+w]
        name, conf = recognize(face)

        if name != "Unknown":
            mark_attendance(name)

        label = f"{name} ({conf:.2f})"
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)
        cv2.putText(frame,label,(x,y-10),
                    cv2.FONT_HERSHEY_SIMPLEX,0.8,(0,255,0),2)

    cv2.imshow("Attendance System", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()